# Most vote(4th) Code Review

**< 학습 코드 내용 >**

notebook based on

SAMUEL https://www.kaggle.com/code/muelsamu/simple-tabpfn-approach-for-score-of-15-in-1-min

@nomuraryota https://www.kaggle.com/nomuraryota work

- SAMUEL의 파일에 기초로 작성하였음

The approach contains the following components:

1. Use an ensemble of TabPFN and (default) XGBoost
2. Reweight the probabilites to match the balanced log loss used in this competition
3. Use median nan imputation
4. Use the time column from the training data, and use a time (max time in training) + 1 for test
5. Use all four classes provided in greeks.Alpha and aggregate probabilites for the latter three classes, as they all correspond to different illnesses

**내용 해석**

1. **TabPFN과 XGBoost를 사용**
2. **이 경쟁에서 사용된 균형잡힌 로그 손실과 일치하도록 확률을 재가중합니다.**
3. **NaN값을 대체하기 위해 중앙값 사용**
4. **Training data에서는 시간열을 사용하고 Test에서는 Training의 최대시간에 +1을 사용**
5. **Greeks에 있는 모든 4개의 Class를 사용할 것.**

In [ ]:
# UNCOMMENT first with internet option turned on
# Use GPU env

# !pip download tabpfn --no-deps -d pip-packages
# !pip install tabpfn
# from tabpfn import TabPFNClassifier
# TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')

# !mv /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt pip-packages/
# !zip -r pip-packages.zip pip-packages

# now you need to download the zip and upload it as dataset with the plus in the top left
# then you need to add it to the notebook as data on the right, and name it `pip-packages-icr`

**내용 해석**

- **인터넷이 켜진 상태에서 주석을 해제하고 GPU를 사용**
1. **TabPFN 다운 및 Classifier 생성**
    - **테이블 형식 데이터 예측을 수행하는 방법을 학습한 신경망 / 사이킷런 인터페이스로 구동할 수 있는 Colab을 개발(SamuelGabriel)**
    - 참고 : https://github.com/automl/TabPFN
2. **Zip파일 다운 후 Data Set을 Kaggle에 업로드**
3. **신규 노트북을 추가하고 이름을 `pip-packages-icr`로 지정 후 아래와 같이 인터넷을 off한 후 아래 처럼 설치 진행할 것.**

In [ ]:
!pip download tabpfn --no-deps -d pip-packages
!pip install tabpfn
from tabpfn import TabPFNClassifier
TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')

!mv /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt pip-packages/
!zip -r pip-packages.zip pip-packages

  File was already downloaded /content/pip-packages/tabpfn-0.1.9-py3-none-any.whl
Successfully downloaded tabpfn
ERROR: Operation cancelled by user
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
mv: cannot stat '/opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt': No such file or directory
updating: pip-packages/ (stored 0%)
updating: pip-packages/tabpfn-0.1.9-py3-none-any.whl (deflated 2%)


In [ ]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages


In [ ]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

cp: cannot stat '/kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt': No such file or directory


In [ ]:
# 데이터 분석에 필요한 라이브러리

import numpy as np
import pandas as pd
import json
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
import xgboost
from tabpfn import TabPFNClassifier
from google.colab import drive

# 표 출력 설
pd.set_option('display.max_columns', None) # 최대열 수를 Max로
pd.set_option('display.max_rows', None) # 최대행 수를 Max로

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# 데이터 파일 주소
drive.mount('./gdrive', force_remount = True) # 구글드라이브로 마운트
BASE_DIR = './gdrive/MyDrive/Colab Notebooks/icr-identify-age-related-conditions' # 기본 파일 주소
train_file = f'{BASE_DIR}/train.csv'
greeks_file = f'{BASE_DIR}/greeks.csv'
test_file = f'{BASE_DIR}/test.csv'

Mounted at ./gdrive


In [ ]:
# 데이터 파일 읽은 후, 데이터 프레임 생성

train_df = pd.read_csv(train_file)
greeks_df = pd.read_csv(greeks_file)
test_df = pd.read_csv(test_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 범주형 Column인 EJ값 및 수량 확인

print(train_df.EJ.unique())

['B' 'A']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 범주형 Column에 따른 0,1로 분류
first_category = train_df.EJ.unique()[0]
train_df.EJ = train_df.EJ.eq(first_category).astype('int')
test_df.EJ = test_df.EJ.eq(first_category).astype('int')

In [ ]:
# Greeks 데이터에서는 사용할 수 있는 시간 정보가 포함되어 있으며 int/NaN으로 구분 분석 가능

from datetime import datetime
times = greeks_df.Epsilon.copy() # Greeks 데이터에서 Epsilon(데이터가 수집된 날짜) 복사

# 날짜, 시간 데이터에서 문자열 데이터로 변환(1년 1월 1일 이후 누적 날짜 수)
times[greeks_df.Epsilon != 'Unknown'] = greeks_df.Epsilon[greeks_df.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeks_df.Epsilon == 'Unknown'] = np.nan

In [ ]:
# 예측 변수 및 대상 열 설정

target_column = 'Class'

# 'Class' 및 'id'열을 제외한 모든 열
predictor_columns = [n for n in train_df.columns if n != target_column and n != 'Id']

In [ ]:
# 앙상블 Class

class WeightedEnsemble(BaseEstimator):
    # 호출 함수
    # self.imputer : 데이터 집합에서 누락된 데이터를 처리(누락된 값을 평균, 중앙값 또는 비번한 값 사용)
    def __init__(self):
        self.classifiers = [xgboost.XGBClassifier(), TabPFNClassifier(N_ensemble_configurations=64, device='cuda:0')]
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')

    # 훈련 함수
    def fit(self, X, y):
        unique_classes, y = np.unique(y, return_inverse=True)
        self.classes_ = unique_classes # A,B,C,D
        X = self.imputer.fit_transform(X) # 결측치 조치 후 변환
        for classifier in self.classifiers:
            classifier.fit(X, y)

    def predict_proba(self, X):
        X = self.imputer.transform(X) # 결측치에 대해 처리
        probabilities = np.stack([classifier.predict_proba(X) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0) # 확률값 평균
        class_0_est_instances = averaged_probabilities[:, 0].sum() # 클래스 0에 대한 확률 합
        others_est_instances = averaged_probabilities[:, 1:].sum() # 클래스 나머지에 대한 확률 합
        # Class 불균형에 기반한 가중 확률
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)

**Class 보충 내용**

* Class : 데이터나 처리의 정의 등을 하나로 정리해둔 모형
    - 장점
        - Global 변수를 없애고, 모든 변수를 어떠한 Scope에 소속시킴
        - 몇 번이고 재사용 가능
        - 코드의 수정을 최소화
        - 함수 실행중에, 함수 자신을 다시 호출하는 처리등이 가능
    - Instance : Class에 정의된 데이터나 함수를 사용하기 위해 생성해야함
        - 하나의 Class에 대해서 Instance는 여러 개 생성하는 것 가능
        - 각각의 Instance에 각각 다른 데이터를 가지도록 할 수 있음
    - Method : Class내에 기재되어 있는 함수를 뜻함
    - Self : Class의 구성을 취득할 때에 정형의 구문
    - __init__
        - Constructor라고 불리는 초기화를 위한 함수(Method)
        - Instance화를 실시할 때 반드시 처음에 호출되는 특수한 함수
        - Object 생성(Instance를 생성)과 관련하여 데이터의 초기를 실시하는 함수

**앙상블 보충내용**

- 앙상블 : 여러 개의 머신러닝 모델을 이용해 최적의 답을 찾아내는 기법
    - 여러 모델을 이용하여 데이터를 학습하고, 모든 모델의 예측결과를 평균하여 예측)
- 앙상블 기법의 종류
    - 보팅(Voting) : 투표를 통해 결과 도출
    - 배깅(Bagging) : 샘플 중복 생성을 통해 결과 도출
    - 부스팅(Boosting) : 이전 오차를 보완하면서 가중치 부여
    - 스태킹(Stacking) : 여러 모델을 기반으로 예측된 결과를 통해 meta 모델이 다시 한번 예측
- 부스팅(Boosting) : 약한 학습기를 순차적으로 학습을 하되, 이전 학습에 대하여 잘못 예측된 데이터에 가중치를 부여해 오차를 보완
    - 순차적인 학습이기에 다른 앙상블 대비 학습 시간이 오래 걸림
- XGBoost(eXtreme Gradient Boosting)
    - 빠른 속도, 확장성, 이식성이 특징
    - GBM(Gradient Boosting Machine)보다 빠르고 향상됨
    - 파이썬 wrapper와 사이킷런 wrapper 두 가지 방법으로 구현가능
        1. Filter Method : 어떤 통계량(주로 상관계수)로 변수 선택
        2. Wrapper Method : 머신러닝 알고리즘에 변수 선택 / 성능 측정 및 반복하여 변수를 잡아가는 것
        3. Embedded Method : 예측 및 최적화 모델링 안에서 변수가 결정

In [ ]:
# Train 데이터에서 예측 변수 및 시간 결합 / Test 데이터에서 예측 변수 및 시간 Zero화 후 Train 시간에 +1시간
train_pred_and_time = pd.concat((train_df[predictor_columns], times), axis=1)
test_predictors = np.array(test_df[predictor_columns])
test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

In [ ]:
# 모델 생성 및 훈련
model = WeightedEnsemble()
model.fit(np.array(train_pred_and_time), np.array(greeks_df['Alpha']))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [ ]:
# 예측 확률
probabilities = model.predict_proba(test_pred_and_time)
assert (model.classes_[0] == 'A')
probabilities = np.concatenate((probabilities[:,:1], np.sum(probabilities[:,1:], 1, keepdims=True)), axis=1)

In [ ]:
# 'A'의 확률값에 대한 범위 지정
p0 = probabilities[:,:1]
p0[p0 > 0.86] = 1
p0[p0 < 0.14] = 0

In [ ]:
submission = pd.DataFrame(test_df["Id"], columns=["Id"])
submission["class_0"] = p0
submission["class_1"] = 1 - p0
submission.to_csv('submission.csv', index=False)

In [ ]:
submission_df = pd.read_csv('submission.csv')
submission_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5
